In [2]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType

model_file = "onnx_models/multiply_model_split.onnx"
model = ModelWrapper(model_file)
model.set_tensor_datatype('281',DataType["INT8"])
# model.set_tensor_datatype('282',DataType["INT8"])
model.set_tensor_datatype('280',DataType["INT8"])
# print(model)
ModelWrapper.save(model,"onnx_models/multiply_model_split_fixed_quant.onnx")

In [3]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "onnx_models/multiply_model_split_fixed_quant.onnx"

final_output_dir = "output_final"

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    board               = "KV260_SOM",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

Previous run results deleted!


In [4]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from onnx_models/multiply_model_split_fixed_quant.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in output_final
Build log is at output_final/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
Running step: step_streamline [3/17]
Running step: step_convert_to_hls [4/17]


Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 426, in step_hls_codegen
    model = model.transform(
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/finn/src/finn/transformation/fpgadataflow/prepare_ip.py", line 88, in apply
    _codegen_single_node(node, model, self.fpgapart, self.clk)
  File "/workspace/finn/src/finn/transformation/fpgadataflow/prepare_ip.py", line 55, in _codegen_single_node
    inst.code_generation_ipgen(model, fpgapart, clk)
  File "/workspace/finn/src/finn/custom_op/fpgadataflow/hlscustomop.py", line 271, in code_generation_ipgen
    self.generate_params(model, path)
  File "/workspace/finn/src/finn/custom_op/fpgadataflow/matrixvectoractiva

Running step: step_create_dataflow_partition [5/17]
Running step: step_target_fps_parallelization [6/17]
Running step: step_apply_folding_config [7/17]
Running step: step_generate_estimate_reports [8/17]
Running step: step_hls_codegen [9/17]
> /workspace/finn/src/finn/util/data_packing.py(84)array2hexstring()
     82     for val in array:
     83         # ensure that this value is permitted by chosen dtype
---> 84         assert dtype.allowed(val), "This value is not permitted by chosen dtype."
     85         if dtype.is_integer():
     86             if dtype.signed():

ipdb> exit
Build failed
CPU times: user 278 ms, sys: 58.5 ms, total: 337 ms
Wall time: 7.71 s


-1